## Computational psychrometric analysis of HVAC system in Abstract model of Roche Tower 1

03.07.2023 , Winterthur

Jeroen Stenzel 

Lorenzo Zgraggen 

Barbara Klemensowska


### Vissualization of the project

![page1](./page1.png)
> Figure 1. Digital Energy twin in context various Air Change Rate 



### Element of the system 

![page2](./page2.png)
> Figure 2. IDA Model Lumped capacitance Modell 




### Introduction to the Modell

![page3](./page3.png)
> Figure 3. Abstract model of Roche Tower 1



**Objectives:**
- Defining the thermodynamical process of HVAC and controlling parameters in thermal zone for the Roche Tower in winter and summer condittions
- Calculating the mass flow rate of supply air 
- Controlling the target conditions in the building 

## Model representing CAV system for a single termal zone with AHU for heating and vapor humidification with recycled air

To ensure that our system distribute air for comfort and safety we created model describing our system: 

## Winter conditions

### Description of the building

![WINTER model basia](./WINTER_model_basia.png)
> Figure 1. Model for Winter conditions 



We use the module based on Va_hum script

In [16]:
import numpy as np
import pandas as pd
import psychro as psy
import matplotlib.pyplot as plt



We determine the design condition of the modell

In [17]:
# Design conditions for CAV (to determine m)
θOd = -1        # °C, outdoor temperarture
θId = 18        # °C, indoor temperature
θSd = 30        # °C summply air temperature
mid = 0.0057    # kg/s, infiltration
Qsad = 216      # kW, sensible auxiliary load
Qlad = 145.2    # kW, latent auxliary load
mid = 0.0057    # kg/s, air infiltration mass flow rate
UAd = 85.9      # kJ/K, overall heat conductance

# constants
c = 1e3         # air specific heat J/kg K
l = 2496e3      # latent heat J/kg

In [18]:
def ModelAllOutAir(m, θS, θIsp, φIsp, θO, φO, Qsa, Qla, mi, UA):
    Kt, Kw = 1e10, 1e10             # controller gain
    wO = psy.w(θO, φO)              # outdoor mumidity ratio
    wIsp = psy.w(θIsp, φIsp)        # indoor mumidity ratio
# Model
    A = np.zeros((10, 10))          # coefficents of unknowns
    b = np.zeros(10)                # vector of inputs
    # HC heating coil
    A[0, 0], A[0, 6], b[0] = m * c, -1, m * c * θO
    A[1, 1], b[1] = m * l, m * l * wO
    # VA vapor humidifier
    A[2, 0], A[2, 2], b[2] = -m * c, m * c, 0
    A[3, 1], A[3, 3], A[3, 7], b[3] = -m * l, m * l, -1, 0
    # TZ thermal zone
    A[4, 2], A[4, 4], A[4, 8], b[4] = -m * c, m * c, -1, 0
    A[5, 3], A[5, 5], A[5, 9], b[5] = -m * l, m * l, -1, 0
    # BL building
    A[6, 4], A[6, 8], b[6] = UA + mi * c, 1, (UA + mi * c) * θO + Qsa
    A[7, 5], A[7, 9], b[7] = mi * l, 1, mi * l * wO + Qla
    # Kt indoor temperature controller
    A[8, 4], A[8, 6], b[8] = Kt, 1, Kt * θIsp
    # Kw indoor hum.ratio controller
    A[9, 5], A[9, 7], b[9] = Kw, 1, Kw * wIsp

    # Solution
    x = np.linalg.solve(A, b)
    return x

We consider the case with Constant Air Volume

In [14]:
def RecAirCAV(α=0.5, θS=30, θIsp=18, φIsp=0.5, θO=-1, φO=1,
              Qsa=0, Qla=0, mi=2.12, UA=935.83):
    """
    CAV Constant Air Volume:
    mass flow rate calculated for design conditions
    maintained constant in all situations
    INPUTS:
        α mixing ratio of outdoor air
        θS    supply air °C
        θIsp  indoor air setpoint °C
        φIsp -
        θO    outdoor temperature for design °C
        φO  outdoor relative humidity for design -
        Qsa   aux. sensible heat W
        Qla   aux. latente heat W
        mi    infiltration massflow rate kg/s
        UA    global conductivity bldg W/K

    System:
        HC:     Heating Coil
        VH:     Vapor Humidifier
        TZ:     Thermal Zone
        Kw:     Controller - humidity
        Kt:     Controller - temperature
        o:      outdoor conditions

    12 Unknowns
        0, 1, 2, 3 points (temperature, humidity ratio)
        QsHC, QlVH, QsTZ, QlTZ

    <-3--|<-------------------------|
         |                          |
    -o->MX--0->HC--1->VH--2->TZ--3-->
               /       /     ||  |
               |       |     BL  |
               |       |         |
               |       |_____Kw__|_w3
               |_____________Kt__|_t3
    """
    plt.close('all')
    wO = psy.w(θO, φO)            # hum. out

    # Mass flow rate for design conditions
    # Supplay air mass flow rate
    # QsZ = UA*(θO - θIsp) + mi*c*(θO - θIsp)
    # m = - QsZ/(c*(θS - θIsp)
    # where
    # θOd, wOd = -1, 3.5e-3           # outdoor
    # θS = 30                       # supply air
    # mid = 2.18                     # infiltration
    QsZ = UA * (θOd - θIsp) + mid * c * (-1 - θIsp) + Qsa
    m = - QsZ / (c * (θS - θIsp))
    print('Winter Recirculated_air CAV')
    print(f'm = {m: 5.3f} kg/s constant (from design conditions)')
    print(f'Design conditions θS = {θS: 3.1f} °C,'
          f'mi = {mid:3.1f} kg/s, θO = {θOd:3.1f} °C, '
          f'θI = {θIsp:3.1f} °C')

    # Model
    x = ModelRecAir(m, α, θS, θIsp, φIsp, θO, φO, Qsa, Qla, mi, UA)
    # (m, θS, mi, θO, φO, α)

    # Processes on psychrometric chart
    # Points      o    0    1   2   3       Elements
    #             0    1    2   3   4
    A = np.array([[-1, 1, 0, 0, -1],        # MX
                 [0, -1, 1, 0, 0],          # HC
                 [0, 0, -1, 1, 0],          # VH
                 [0, 0, 0, -1, 1]])         # TZ
    t = np.append(θO, x[0:8:2])

    print(f'wO = {wO:6.5f}')
    w = np.append(wO, x[1:8:2])
    psy.chartA(t, w, A)

    t = pd.Series(t)
    w = 1000 * pd.Series(w)
    P = pd.concat([t, w], axis=1)       # points
    P.columns = ['θ [°C]', 'w [g/kg]']

    output = P.to_string(formatters={
        'θ [°C]': '{:,.2f}'.format,
        'w [g/kg]': '{:,.2f}'.format
    })
    print()
    print(output)

    Q = pd.Series(x[8:], index=['QsHC', 'QlVH', 'QsTZ', 'QlTZ'])
    pd.options.display.float_format = '{:,.2f}'.format
    print()
    print(Q.to_frame().T / 1000, 'kW')

    return None

To personalise desired parameters we implement foloowing widgets:

In [15]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import va_hum

# %matplotlib inline  # uncomment for inline figure
# uncomment for figure in separate window
# %matplotlib qt
# plt.show()

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)





def RecAirCAV_wd(α=0.5, θS=30, θIsp=18, φIsp=0.5, θO=-1, φO=1):
    Qsa = 0.
    Qla = 0.
    mi = 1
    UA = 935.83
    from va_hum import RecAirCAV
    RecAirCAV(α, θS, θIsp, φIsp, θO, φO, Qsa, Qla, mi, UA)



interact(RecAirCAV_wd, α = (0, 1, 0.1), θS = (20, 50, 2),
    θIsp = (17, 25, 1), φIsp = (0, 1, 0.1),
    θO = (-10., 17., 2), φO = (0, 1, 0.1));

# After canges to the building
# uncomment following 15 lines to have two graphs

def RecAirCAV_wd(Qsa=0, Qla=0, mi=2.12, UA=935.83):
    α = 0.5
    θSsp = 30
    θIsp = 15
    φIsp = 0.5
    θO = -1
    φO = 1
    from va_hum import RecAirCAV
    RecAirCAV(α, θSsp, θIsp, φIsp, θO, φO, 
              Qsa, Qla, mi, UA);


'''
interact(RecAirCAV_wd, Qsa=(0, 15000, 50), Qla=(0, 15000, 50),
         mi=(0, 5, 0.2), UA=(700, 1000, 10));
'''

interactive(children=(FloatSlider(value=0.5, description='α', max=1.0), IntSlider(value=30, description='θS', …

'\ninteract(RecAirCAV_wd, Qsa=(0, 15000, 50), Qla=(0, 15000, 50),\n         mi=(0, 5, 0.2), UA=(700, 1000, 10));\n'

## Summer conditions

### Description of the building

![summer_model](./summer_model.png)

> Figure 2. Model for Summer conditions


In [19]:
# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat

# to be used in self.m_ls / least_squares
m_max = 100     # ks/s, max dry air mass flow rate
θs_0 = 5        # °C, initial guess for saturation temperature

In [20]:
 def lin_model(self, θs0):
        m, mo, β, Kθ, Kw, θo, φo, θIsp, φIsp, mi, UA, Qsa, Qla = self.actual
        wo = psy.w(θo, φo)      # hum. out

        A = np.zeros((14, 14))  # coefficents of unknowns
        b = np.zeros(14)        # vector of inputs
        # MX1
        A[0, 0], A[0, 8], b[0] = m * c, (m - mo) * c, mo * c * θo
        A[1, 1], A[1, 9], b[1] = m * l, (m - mo) * l, mo * l * wo
        # CC
        A[2, 0], A[2, 2], A[2, 11], b[2] = m * c, - m * c,\
            1, 0
        A[3, 1], A[3, 3], A[3, 12], b[3] = m * l, - m * l,\
            1, 0
        A[4, 2], A[4, 3], b[4] = psy.wsp(θs0), -1,\
            psy.wsp(θs0) * θs0 - psy.w(θs0, 1)
        A[5, 10], A[5, 11], A[5, 12], b[5] = -1, 1, 1, 0
        # HC
        A[6, 2], A[6, 4], A[6, 11], b[6] = m * c, -m * c, 1, 0
        A[7, 3], A[7, 5], b[7] = m * l, -m * l, 0
        # TZ
        A[8, 4], A[8, 6], A[8, 12], b[8] = m * c, -m * c, 1, 0
        A[9, 5], A[9, 7], A[9, 13], b[9] = m * l, -m * l, 1, 0
        # BL
        A[10, 6], A[10, 12], b[10] = (UA + mi * c), 1, (UA + mi * c) * θo + Qsa
        A[11, 7], A[11, 13], b[11] = mi * l, 1, mi * l * wo + Qla
        # Kθ indoor temperature controller
        A[12, 6], A[12, 8], b[12] = Kθ, 1, Kθ * θIsp
        # Kw indoor humidity ratio controller
        A[13, 7], A[13, 11], b[13] = Kw, 1, Kw * psy.w(θIsp, φIsp)
        x = np.linalg.solve(A, b)
        return x

In [21]:
    def solve_lin(self, θs0):
        """
        Finds saturation point on saturation curve ws = f(θs).
            Solves iterativelly *lin_model(θs0)*:
            θs -> θs0 until ws = psy(θs, 1).

        Parameters
        ----------
        θs0     initial guess saturation temperature

        Method from object
        ---------------------
        *self.lin_model(θs0)*

        Returns (16 unknowns)
        ---------------------
        x of *self.lin_model(self, θs0)*
        """
        Δ_ws = 10e-3  # kg/kg, initial difference to start the iterations
        while Δ_ws > 0.01e-3:
            x = self.lin_model(θs0)
            Δ_ws = abs(psy.w(x[2], 1) - x[3])   # psy.w(θs, 1) = ws
            θs0 = x[2]                          # actualize θs0
        return x

In [22]:
import ipywidgets as wd
import matplotlib.pyplot as plt
import cool_summer as cc
import psychro as psy
# %matplotlib inline  # uncomment for inline figure
# %matplotlib qt      # uncomment for figure in separate window
# plt.show()

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)

# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat
ρ = 1.2         # kg/m3, density

# Building dimensions
length = 12     # m
width = 12      # m
height = 3    # m
persons = 15   # m

sens_heat_person = 60       # W / person
latent_heat_person = 40     # W / person
load_m2 = 23       # W/m2
solar_m2 = 200      # W/m2 of window area
ACH = 0.15             # Air Cnhnages per Hour
U_wall = 0.7        # W/K, overall heat transfer coeff. walls
U_window = 0.7      # W/K, overall heat transfer coeff. windows

θo, φo = 30, 0.5    # outdoor temperature & relative humidity
θI, φI = 26, 0.5    # indoor temperature & relative humidity
wo = psy.w(θo, φo)
wI = psy.w(θI, φI)

floor_area = length * width
surface_floor = 2 * (length + width) * height + floor_area
surface_wall = 0.9 * surface_floor
surface_window = surface_floor - surface_wall

UA = U_wall * surface_wall + U_window * surface_window
mi = ACH * surface_floor * height / 3600 * ρ

solar_gains = solar_m2 * surface_window
electrical_load = load_m2 * surface_floor
Qsa = persons * sens_heat_person + solar_gains + electrical_load
Qla = persons * latent_heat_person

QsTZ = (UA + mi * c) * (θo - θI) + Qsa
QlTZ = mi * l * (wo - wI) + Qla

θS = θI - 15        # °C supply air temperature
m = QsTZ / c / (θI - θS)

print(f'QsTZ = {QsTZ:.0f} W, QlTZ = {QlTZ:.0f} W')
print(f'UA = {UA:.0f} W/K, mi = {mi:.2f} kg/s,\
      Qsa = {Qsa:.0f} W, Qla = {Qla:.0f} W')
print(f'm = {m:.3f} kg/s')

Kθ, Kw = 1e10, 0     # Kw can be 0
β = 0.2              # by-pass factor

m, mo = 3.5, 1.      # kg/s, mass flow rate: supply & outdoor (fresh) air
θo, φo = 30., 0.5    # outdoor conditions
θIsp, φIsp = 26., 0.5        # set point for indoor condition

mi = 1.35            # kg/s, mass flow rate of infiltration air
UA = 675.            # W/K, overall heat coefficient of the building
Qsa, Qla = 34000., 4000.     # W, auxiliary loads: sensible & latent

parameters = m, mo, β, Kθ, Kw
inputs = θo, φo, θIsp, φIsp, mi, UA, Qsa, Qla


cool6 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 1e10
cool6.actual[4] = Kw
wd.interact(cool6.VAV_wd, value='θS', sp=(14, 21), θo=(18, 30), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));



QsTZ = 14263 W, QlTZ = 903 W
UA = 202 W/K, mi = 0.04 kg/s,      Qsa = 13284 W, Qla = 600 W
m = 0.951 kg/s


interactive(children=(Text(value='θS', description='value'), IntSlider(value=18, description='sp', max=21, min…

In [25]:
'''
interact(RecAirCAV_wd, α = (0, 1, 0.1), θS = (30, 50, 2),
    θIsp = (17, 25, 1), φIsp = (0, 1, 0.1),
    θO = (20, 35., 2), φO = (0, 1, 0.1));
'''

'\ninteract(RecAirCAV_wd, α = (0, 1, 0.1), θS = (30, 50, 2),\n    θIsp = (17, 25, 1), φIsp = (0, 1, 0.1),\n    θO = (20, 35., 2), φO = (0, 1, 0.1));\n'